# Mobility-Station-Finder

## Imports

In [1]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from matrixconverters.read_ptv import ReadPTVMatrix
from collections import namedtuple
import requests

## Read and process static data

### Paths

In [2]:
path_npvm_zones = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\Verkehrszonen_Schweiz_NPVM_2017_shp.zip"
path_mobility_stations = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\mobility-stationen-und-fahrzeuge-schweiz.csv"
path_pt_jrta = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\140_JRTA_(OEV).mtx"
path_pt_ntr = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\144_NTR_(OEV).mtx"

### Read NPVM-zones with shapes

In [45]:
%time gdf_npvm_zones = gpd.read_file(path_npvm_zones, encoding="cp1252").to_crs(4326)

CPU times: total: 3.38 s
Wall time: 3.36 s


In [4]:
gdf_npvm_zones.head()

,ID,ID_alt,ID_Gem,N_Gem,stg_type,N_stg_type,ID_KT,N_KT,ID_SL3,N_SL3,ID_Agglo,N_Agglo,ID_AMR,N_AMR,geometry
0,101001,1,1,Aeugst am Albis,1,None,1,ZH,3,Ländlich,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.47334 47.26128, 8.47334 47.26139, ..."
1,201001,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.42224 47.29775, 8.42282 47.29816, ..."
2,201002,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.44770 47.26794, 8.44767 47.26782, ..."
3,201003,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.43834 47.27714, 8.43814 47.27726, ..."
4,201004,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.45000 47.27949, 8.45007 47.27945, ..."


In [5]:
print("Anzahl NPVM-Zonen: {}".format(len(gdf_npvm_zones)))

Anzahl NPVM-Zonen: 7978


In [6]:
def get_npvm_zone(id_):
    return gdf_npvm_zones[gdf_npvm_zones.ID == id_]

In [7]:
get_npvm_zone(5301003)

,ID,ID_alt,ID_Gem,N_Gem,stg_type,N_stg_type,ID_KT,N_KT,ID_SL3,N_SL3,ID_Agglo,N_Agglo,ID_AMR,N_AMR,geometry
84,5301003,53,53,Bülach,1,None,1,ZH,1,Städtisch,261,Zürich,12032,Kloten,"POLYGON ((8.53973 47.51618, 8.54007 47.51529, ..."


### Read Mobility-stations, assign NPVM-zones to Mobility-stations

In [8]:
df_mobility_vechicles = pd.read_csv(path_mobility_stations, delimiter=";", encoding="utf8")[["Stationsnummer", "Name", "Standort"]].dropna()

In [12]:
df_mobility_vechicles["lon"] = df_mobility_vechicles["Standort"].apply(lambda x: x.split(",")[1])
df_mobility_vechicles["lat"] = df_mobility_vechicles["Standort"].apply(lambda x: x.split(",")[0])
gdf_mobility_vechicles = gpd.GeoDataFrame(df_mobility_vechicles, geometry=gpd.points_from_xy(df_mobility_vechicles.lon, df_mobility_vechicles.lat), crs=4326)

In [13]:
print("Anzahl Mobility Fahrzeuge: {}".format(len(df_mobility_vechicles)))

Anzahl Mobility Fahrzeuge: 2662


In [63]:
gdf_mobilty_vehicles_with_npvm_zone = gpd.sjoin(gdf_mobility_vechicles, gdf_npvm_zones)[["Stationsnummer", "Name", "geometry", "ID", "N_Gem"]]

In [64]:
print("Anzahl Mobility Fahrzeuge mit zugeordneter NPVM-Zone: {}".format(len(gdf_mobilty_vehicles_with_npvm_zone)))

Anzahl Mobility Fahrzeuge mit zugeordneter NPVM-Zone: 2661


In [65]:
gdf_mobilty_vehicles_with_npvm_zone.head()

,Stationsnummer,Name,geometry,ID,N_Gem
0,4500,Aarau Aarenau,POINT (8.04796 47.40060),400101003,Aarau
1072,4016,Aarau Aarepark,POINT (8.03994 47.39647),400101003,Aarau
1,1509,Aarau Alte Schoggi-Frey,POINT (8.05569 47.40019),400101012,Aarau
2,1864,Aarau Bahnhof,POINT (8.05288 47.39178),400101008,Aarau
534,1864,Aarau Bahnhof,POINT (8.05288 47.39178),400101008,Aarau


In [119]:
gdf_npvm_zones_with_mobility_station = gdf_mobilty_vehicles_with_npvm_zone[["ID", "N_Gem", "geometry"]].dissolve(by="ID").reset_index()

In [120]:
print("Anzahl NPVM-Zonen mit Mobility-Standort: {}".format(len(gdf_npvm_zones_with_mobility_station)))

Anzahl NPVM-Zonen mit Mobility-Standort: 1311


In [121]:
gdf_npvm_zones_with_mobility_station.head()

,ID,geometry,N_Gem
0,201004,POINT (8.44662 47.27617),Affoltern am Albis
1,201005,POINT (8.45109 47.27595),Affoltern am Albis
2,201009,POINT (8.45641 47.28340),Affoltern am Albis
3,301002,POINT (8.46839 47.32690),Bonstetten
4,301003,POINT (8.47101 47.31595),Bonstetten


### Read PT-skims

In [43]:
%time skim_jrta = ReadPTVMatrix(path_pt_jrta)

CPU times: total: 7.27 s
Wall time: 7.27 s


In [44]:
%time skim_ntr = ReadPTVMatrix(path_pt_ntr)

CPU times: total: 7.64 s
Wall time: 7.66 s


In [47]:
def get_skim(skim_matrix, from_npvm_zone_id, to_npvm_zone_id):
    return skim_matrix.sel(origins=from_npvm_zone_id).sel(destinations=to_npvm_zone_id).matrix.item()

In [48]:
def get_jrta(from_npvm_zone_id, to_npvm_zone_id):
    return get_skim(skim_jrta, from_npvm_zone_id, to_npvm_zone_id)

In [49]:
def get_ntr(from_npvm_zone_id, to_npvm_zone_id):
    return get_skim(skim_ntr, from_npvm_zone_id, to_npvm_zone_id)

In [50]:
get_jrta(378601001, 35101026)

286.0875843780647

In [51]:
get_ntr(378601001, 35101026)

3.000000000000001

## Execute query

### Define origin and destination and assign NPVM-Zone

In [95]:
orig_lon_lat = (7.423570, 46.936620)
dest_lon_lat = (7.695260, 46.828540)

In [96]:
orig_point = Point(orig_lon_lat[0], orig_lon_lat[1])
dest_point = Point(dest_lon_lat[0], dest_lon_lat[1])

In [97]:
gdf_orig = gpd.GeoDataFrame({'geometry': [orig_point]}, crs="EPSG:4326")
gdf_dest = gpd.GeoDataFrame({'geometry': [dest_point]}, crs="EPSG:4326")

In [98]:
gdf_orig_with_zone = gpd.sjoin(gdf_orig, gdf_npvm_zones)[["ID", "N_Gem", "geometry"]]
gdf_dest_zone = gpd.sjoin(gdf_orig, gdf_npvm_zones)[["ID", "N_Gem", "geometry"]]

In [99]:
gdf_orig_with_zone

,ID,N_Gem,geometry
0,35101026,Bern,POINT (7.42357 46.93662)


In [100]:
gdf_dest_zone

,ID,N_Gem,geometry
0,35101026,Bern,POINT (7.42357 46.93662)


In [101]:
orig_zone_id = gdf_orig_with_zone["ID"].item()

### Compute potential NPVM-zones with Mobility-station

In [102]:
gdf_circle_around_orig  = gpd.GeoDataFrame(geometry=gdf_orig_with_zone.to_crs(2026).buffer(5*1000).to_crs(4326), crs="EPSG:4326")
gdf_circle_around_dest = gpd.GeoDataFrame(geometry=gdf_dest_zone.to_crs(2026).buffer(50*1000).to_crs(4326), crs="EPSG:4326")

In [103]:
gdf_circle_around_orig

,geometry
0,"POLYGON ((7.40836 46.90462, 7.40386 46.90580, ..."


In [104]:
gdf_circle_around_dest

,geometry
0,"POLYGON ((7.27261 46.61729, 7.22799 46.62892, ..."


In [105]:
gdf_npvm_zones_with_mobility_station_in_orig_circle = gpd.sjoin(gdf_npvm_zones_with_mobility_station, gdf_circle_around_orig)
gdf_npvm_zones_with_mobility_station_in_dest_circle = gpd.sjoin(gdf_npvm_zones_with_mobility_station, gdf_circle_around_dest)

In [106]:
print("Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Startpunkts: {}".format(len(gdf_npvm_zones_with_mobility_station_in_orig_circle)))
print("Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Zielpunkts: {}".format(len(gdf_npvm_zones_with_mobility_station_in_dest_circle)))

Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Startpunkts: 63
Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Zielpunkts: 225


In [107]:
gdf_npvm_zones_with_mobility_station_in_orig_circle.head()

,ID,geometry,N_Gem,index_right
347,35101004,"MULTIPOINT (7.38055 46.94513, 7.38364 46.94701)",Bern,0
348,35101006,POINT (7.38535 46.93170),Bern,0
349,35101007,POINT (7.38609 46.94300),Bern,0
350,35101011,POINT (7.38905 46.94569),Bern,0
351,35101016,"MULTIPOINT (7.39396 46.94112, 7.39438 46.93700...",Bern,0


In [108]:
gdf_npvm_zones_with_mobility_station_in_orig_circle.head()

,ID,geometry,N_Gem,index_right
347,35101004,"MULTIPOINT (7.38055 46.94513, 7.38364 46.94701)",Bern,0
348,35101006,POINT (7.38535 46.93170),Bern,0
349,35101007,POINT (7.38609 46.94300),Bern,0
350,35101011,POINT (7.38905 46.94569),Bern,0
351,35101016,"MULTIPOINT (7.39396 46.94112, 7.39438 46.93700...",Bern,0


In [124]:
gdf_npvm_zones_with_potential_mobility_station = pd.concat([gdf_npvm_zones_with_mobility_station_in_orig_circle, gdf_npvm_zones_with_mobility_station_in_dest_circle]).groupby("ID").first().reset_index().drop(["index_right"], axis=1)

In [125]:
print("Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Start- oder Zielpunkts: {}".format(len(gdf_npvm_zones_with_potential_mobility_station)))

Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Start- oder Zielpunkts: 225


In [150]:
gdf_npvm_zones_with_potential_mobility_station.head()

,ID,geometry,N_Gem
0,30101004,POINT (7.27780 47.04357),Aarberg
1,30601005,POINT (7.30569 47.07726),Lyss
2,30601007,POINT (7.30560 47.07900),Lyss
3,30601012,POINT (7.31873 47.09890),Lyss
4,31101002,POINT (7.38656 47.04189),Schüpfen


In [129]:
gdf_npvm_zones_with_potential_mobility_station[gdf_npvm_zones_with_potential_mobility_station.ID == 30601007]

,ID,geometry,N_Gem
2,30601007,POINT (7.30560 47.07900),Lyss


### Get Mobility travel time and distance from ORMS

In [145]:
list_npvm_zones_with_potential_mobility_station = list(gdf_npvm_zones_with_potential_mobility_station.to_records())

In [146]:
list_npvm_zones_with_potential_mobility_station[:3]

[(0, 30101004, <POINT (7.278 47.044)>, 'Aarberg'),
 (1, 30601005, <POINT (7.306 47.077)>, 'Lyss'),
 (2, 30601007, <POINT (7.306 47.079)>, 'Lyss')]

In [151]:
coords_str = "{},{}".format(dest_point.x, dest_point.y)
for pot_mob_st in list_npvm_zones_with_potential_mobility_station:
    center = pot_mob_st[2].centroid
    coords_str += ";{},{}".format(center.x, center.y)
url = "https://router.project-osrm.org/table/v1/driving/{}?destinations=0&annotations=duration,distance".format(coords_str)

In [152]:
res = requests.get(url).json()

In [183]:
road_distances_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id = {x[1]: res["distances"][x[0] + 1][0] for x in list_npvm_zones_with_potential_mobility_station}
road_durations_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id = {x[1]: res["durations"][x[0] + 1][0] for x in list_npvm_zones_with_potential_mobility_station}

In [169]:
print("Anzahl Distanzen: {}".format(len(road_distances_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id)))
print("Anzahl Reisezeiten: {}".format(len(road_duration_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id)))

Anzahl Distanzen: 225
Anzahl Reisezeiten: 225


In [195]:
pd_distances = pd.DataFrame(list(road_distances_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id.items()), columns=["ID", "MIV_Distanz_bis_Ziel_km"])
pd_distances["MIV_Distanz_bis_Ziel_km"] = pd_distances["MIV_Distanz_bis_Ziel_km"] / 1000.0
pd_durations= pd.DataFrame(list(road_durations_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id.items()), columns=["ID", "MIV_Zeit_bis_Ziel_min"])
pd_durations["MIV_Zeit_bis_Ziel_min"] = pd_durations["MIV_Zeit_bis_Ziel_min"] / 60.0

gdf_npvm_zones_with_potential_mobility_station_distance_duration_to_orig = pd.merge(gdf_npvm_zones_with_potential_mobility_station, pd_dist, on=["ID"])
gdf_npvm_zones_with_potential_mobility_station_distance_duration_to_orig = pd.merge(gdf_npvm_zones_with_potential_mobility_station_distance_duration_to_orig, pd_durations, on=["ID"])


In [196]:
gdf_npvm_zones_with_potential_mobility_station_distance_duration_to_orig.head()

,ID,geometry,N_Gem,Distanz_bis_Ziel_km,MIV_Zeit_bis_Ziel_min
0,30101004,POINT (7.27780 47.04357),Aarberg,63.7240,52.573333
1,30601005,POINT (7.30569 47.07726),Lyss,65.1995,53.448333
2,30601007,POINT (7.30560 47.07900),Lyss,63.7379,53.451667
3,30601012,POINT (7.31873 47.09890),Lyss,68.2743,57.246667
4,31101002,POINT (7.38656 47.04189),Schüpfen,53.7185,46.515000


In [197]:
gdf_npvm_zones_with_potential_mobility_station_distance_duration_to_orig[gdf_npvm_zones_with_potential_mobility_station_distance_duration_to_orig.N_Gem == "Steffisburg"]

,ID,geometry,N_Gem,Distanz_bis_Ziel_km,MIV_Zeit_bis_Ziel_min
171,93901006,POINT (7.63106 46.77080),Steffisburg,9.9043,11.820000
172,93901007,POINT (7.63479 46.78196),Steffisburg,8.6377,10.198333
173,93901010,POINT (7.63644 46.77829),Steffisburg,9.0672,10.671667


In [198]:
gdf_npvm_zones_with_potential_mobility_station_distance_duration_to_orig[gdf_npvm_zones_with_potential_mobility_station_distance_duration_to_orig.N_Gem == "Thun"]

,ID,geometry,N_Gem,Distanz_bis_Ziel_km,MIV_Zeit_bis_Ziel_min
174,94201008,POINT (7.61516 46.73598),Thun,15.1392,20.143333
175,94201009,POINT (7.61651 46.74410),Thun,13.8218,17.933333
176,94201010,POINT (7.61361 46.75667),Thun,12.2229,16.946667
177,94201013,POINT (7.62106 46.73390),Thun,14.7049,18.981667
178,94201014,POINT (7.62483 46.74064),Thun,13.8524,17.673333
179,94201016,"MULTIPOINT (7.62172 46.76000, 7.62823 46.75613)",Thun,12.2399,15.666667
180,94201017,POINT (7.62580 46.74950),Thun,12.6738,16.148333
181,94201018,"MULTIPOINT (7.62289 46.75368, 7.62588 46.75568...",Thun,12.1832,15.658333


In [199]:
gdf_npvm_zones_with_potential_mobility_station_distance_duration_to_orig[gdf_npvm_zones_with_potential_mobility_station_distance_duration_to_orig.N_Gem == "Bern"]

,ID,geometry,N_Gem,Distanz_bis_Ziel_km,MIV_Zeit_bis_Ziel_min
5,35101004,"MULTIPOINT (7.38055 46.94513, 7.38364 46.94701)",Bern,46.4663,41.708333
6,35101006,POINT (7.38535 46.93170),Bern,42.7105,42.688333
7,35101007,POINT (7.38609 46.94300),Bern,46.1887,42.108333
8,35101011,POINT (7.38905 46.94569),Bern,45.8822,42.721667
9,35101016,"MULTIPOINT (7.39396 46.94112, 7.39438 46.93700...",Bern,46.2663,42.521667
10,35101019,"MULTIPOINT (7.40280 46.94723, 7.40677 46.94837)",Bern,45.1186,40.950000
11,35101020,"MULTIPOINT (7.40138 46.94269, 7.40442 46.94609)",Bern,45.3721,40.935000
12,35101022,POINT (7.41138 46.94460),Bern,40.2596,39.006667
13,35101025,POINT (7.41610 46.94553),Bern,39.9281,38.533333
14,35101026,POINT (7.42118 46.93880),Bern,39.2916,37.776667


### Get pt skims from origin to potential Mobility stations

In [ ]:
zone_ids_list = list(potential_mobility_stations[["ID"]].to_records())
jrta_list = [get_jrta(orig_zone_id, x[1]) for x in zone_ids_list]
ntr_list = [get_ntr(orig_zone_id, x[1]) for x in zone_ids_list]

In [ ]:
zone_ids_list[:5]

In [ ]:
print(len(jrta_list))
print(len(ntr_list))

In [ ]:
jrta_list[:5]

In [ ]:
ntr_list[:5]

TODO:
- Calculate generalized costs

### Visualize situation on map

In [ ]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl
import geopandas
import json


m = Map(center=(52.3,8.0), zoom = 3, basemap= basemaps.Esri.WorldTopoMap)

geo_data = GeoData(geo_dataframe = df_npvm_zones,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Countries')

m.add_layer(geo_data)
m.add_control(LayersControl())

In [ ]:
m.layout.width = '100%'
m.layout.height = '1000px'
m
# m.save("npvm-zonen.html")